In [1]:
# Usuals imports 
import numpy as np
import pandas as pd
from time import sleep
import re

# df_productor = df_productor.explode('idTitles').reset_index().drop(columns=['index'])

# df_imdb = pd.merge(df_imdb_fr_noted, df_productor, left_on='titleId', right_on='idTitles')

# cities_weather_infos = pd.read_csv('src/cities_weather_infos.csv')

# 1. cities_weather_info.ipynb create from a list of cities a dataframe with weather of this cities.
# 2. weather_code.py return a dataframe who associate code with a definition of this code.
# 3. booking_infos.ipynb return the first page of city result for current date.
# 4. booking_infos_pages.py return the numbers of results availables on firsts pages.
# 5. booking_hostels.py make a list of all hostels availables for previous list of firts pages.
# 6. hostels_list.py make a list of all hostels forms from hotels cities list.
# 7. hostels_list_located.ipynb add locations to this last dataset.

# 1 -> cities_weather_infos.csv
# 2 -> weather_code_description.json : weather_code <-> cities_weather_infos.csv
# 3 -> booking_mains_cities.csv : city <-> cities_weather_infos.csv
# 5 -> booking_hostels.json : booking_page <-> +- city : booking_mains_cities.csv
# 7 -> hostels_list_located.csv : booking_page <-> url_hostel 



In [48]:
hostels_list_located = pd.read_csv('src/hostels_list_located.csv')
booking_hostels = pd.read_json('src/booking_hostels.json')
booking_index = pd.read_json('src/booking_index.json')
booking_mains_cities = pd.read_csv('src/booking_mains_cities.csv')

weather_code_description = pd.read_json('src/weather_code_description.json')
cities_weather_infos = pd.read_csv('src/cities_weather_infos.csv')

In [54]:
dataframe = pd.merge(cities_weather_infos, weather_code_description, left_on='weather_code', right_on='weather_code')
dataframe = pd.merge(dataframe, booking_mains_cities, left_on='city', right_on='city')

display(dataframe[5:10])
print(len(dataframe))

#display(hostels_list_located[5:10])
#print(len(hostels_list_located))

,city,latitude,longitude,adress,weather_code,felt_temperature,cloud_coverage,precipitation,wind,description,main_url
5,Montauban,44.017584,1.354999,"Montauban, Tarn-et-Garonne, Occitanie, France ...",802,22.1,60,0.0,3.48718,Scattered clouds,https://www.booking.com/searchresults.fr.html?...
6,St Malo,48.649518,-2.026041,"Saint-Malo, Ille-et-Vilaine, Bretagne, France ...",800,17.0,0,0.0,4.60000,Clear sky,https://www.booking.com/searchresults.fr.html?...
7,Strasbourg,48.584614,7.750713,"Strasbourg, Bas-Rhin, Grand Est, France métrop...",800,22.0,0,0.0,2.10000,Clear sky,https://www.booking.com/searchresults.fr.html?...
8,Colmar,48.077752,7.357964,"Colmar, Colmar-Ribeauvillé, Haut-Rhin, Grand E...",800,21.1,0,0.0,1.10000,Clear sky,https://www.booking.com/searchresults.fr.html?...
9,Eguisheim,48.044797,7.307962,"Eguisheim, Colmar-Ribeauvillé, Haut-Rhin, Gran...",800,20.6,0,0.0,2.21187,Clear sky,https://www.booking.com/searchresults.fr.html?...


35


In [ ]:
hostels_list_located = hostels_list_located.assign(city='',latitude=0.0,
                                                   longitude=0.0, adress='',
                                                   weather_code=0, felt_temperature=0.0,
                                                   cloud_coverage=0, precipitation=0.0,
                                                   wind=0.0, description='', main_url=''
                                                   )

for item in range(len(hostels_list_located)):
    pass
